# Analysing the data

Getting insights from the complete and tidy dataset to plot into graphs on **Datawrapper**.

In [1]:
# Import libraries
import pandas as pd
import warnings

In [2]:
# Open dataset
df = pd.read_csv("../04_tidy_data/china_cleaned.csv")
df.head()

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
0,2025-05-11,Beijing,Taxi from the airport to Renata's house,Carol,apps,Transportation,87.00,12.08,6.04
1,2025-05-12,Beijing,Didi to the fried dumpling restaurant,Carol,apps,Transportation,49.93,6.93,3.47
2,2025-05-12,Beijing,Necklace and earrings Carol + gift earring fro...,Carol,apps,Shopping,170.00,23.61,11.81
3,2025-05-12,Beijing,3 Decks of cards,Carol,apps,Shopping,90.00,12.50,6.25
4,2025-05-12,Beijing,Subway,Carol,apps,Transportation,6.00,0.83,0.42


## 1- Comparing expenses in general

#### How much was spent in total per capita?

In [3]:
total_spent = df["price_usd_per_capita"].sum()
print(f"The total spent per person on the entire trip was", total_spent, "dollars.")

The total spent per person on the entire trip was 4452.59 dollars.


#### What were our top 10 largest unique expenses?

In [4]:
top10 = df.sort_values(by="price_usd_per_capita", ascending=False).nlargest(10, "price_usd_per_capita")
top10

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
117,2025-05-11,China,Round-trip flight from São Paulo to Beijing,Paula,credit card,Transportation,23791.90,3304.43,1652.21
114,2025-05-24,Lhasa,Tibet tour package,Paula,credit card,Tour Agency,15183.00,2108.75,702.92
112,2025-05-24,Lhasa,Round-trip flight from Beijing to Lhasa,Paula,credit card,Transportation,9169.11,1273.49,424.50
149,2025-05-31,Guangzhou,Round-trip flight from Beijing to Guangzhou,Renata,apps,Transportation,6150.00,854.17,284.72
87,2025-05-20,Shanghai,Flight from Shanghai to Beijing,Renata,apps,Transportation,1460.00,202.78,101.39
84,2025-05-19,Shanghai,Uniqlo haul,Paula,credit card,Shopping,1162.00,161.39,80.69
147,2025-05-30,Beijing,Silk scarves,Paula,credit card,Shopping,1145.00,159.03,79.51
15,2025-05-13,Datong,Datong tourism package,Paula,credit card,Tour Agency,1080.00,150.00,75.00
70,2025-05-18,Shanghai,Homeinn Hotel,Renata,apps,Hotel,970.00,134.72,67.36
148,2025-05-31,Guangzhou,SunYat Sen University Kaifeng Hotel,Renata,apps,Hotel,1382.00,191.94,63.98


Predictable: **airfares** are the more expensive thing, and longer international flights are on top.
<br>
The **tour packages** (especially the farthest and longest one, to Tibet), are also there.
<br>
And then we have some more expensive **hotels**... and *two specific purchases* that stand out and we can explore in the dataviz.
<br>
Out of curiosity, let's first find out how big of a chunk was the international airfare:

In [5]:
expenses = df.nlargest(1, "price_usd_per_capita")
expenses = expenses[["price_usd_per_capita"]].rename(columns={ "price_usd_per_capita" : "airfare"})
expenses

,airfare
117,1652.21


In [6]:
expenses.insert(loc=1, column="total", value=total_spent)

In [7]:
expenses["others"] = expenses["total"] - expenses["airfare"]
expenses["pct_airfare"] = (expenses["airfare"] / expenses["total"]).round(3)
expenses

,airfare,total,others,pct_airfare
117,1652.21,4452.59,2800.38,0.371


Ok, getting there and back took more than two thirds of all the costs. It's a good indicator that domestic transportation, hotels and food might not be super expensive. (Not to mention we didn't need to pay for a hotel in Beijing).

In [8]:
# Saving this as a CSV to use on Datawrapper for a donut chart
expenses_to_chart = expenses[["airfare", "others"]]
expenses_to_chart.to_csv("../04_tidy_data/chart_00_pct_airfare.csv", index=False)
expenses_to_chart

,airfare,others
117,1652.21,2800.38


#### What about the total cost for each category?

In [9]:
# Group by category and sum the total spent in each of them
by_category = df.groupby("category")["price_usd_per_capita"].sum().reset_index()
by_category = by_category.sort_values(by="price_usd_per_capita", ascending=False)
by_category

,category,price_usd_per_capita
5,Transportation,2711.34
4,Tour Agency,777.92
2,Shopping,400.95
0,Food,225.44
1,Hotel,181.99
3,Tickets,154.95


In [10]:
# Saving this as a CSV to make a histogram chart
by_category.to_csv("../04_tidy_data/chart_00_expenses_by_category.csv", index=False)

#### If the airfare was 37%, where did we spend the rest of the money?

In [11]:
other_expenses = df[df["expense"] != "Round-trip flight from São Paulo to Beijing"]
other_expenses = other_expenses.groupby("category")["price_usd_per_capita"].sum().reset_index()
other_expenses = other_expenses.sort_values(by="price_usd_per_capita", ascending=False)
other_expenses

,category,price_usd_per_capita
5,Transportation,1059.13
4,Tour Agency,777.92
2,Shopping,400.95
0,Food,225.44
1,Hotel,181.99
3,Tickets,154.95


Transportation is still on top.

In [12]:
# Saving this as a CSV to make a histogram chart
other_expenses.to_csv("../04_tidy_data/chart_00_other_expenses_by_category.csv", index=False)

## 2- Comparing expenses over time

#### How much did we spend per person per day?

In [13]:
# Group by date and sum the total spent in each of them
by_day = df.groupby("date")["price_usd_per_capita"].sum().reset_index()

by_day

,date,price_usd_per_capita
0,2025-05-11,1658.25
1,2025-05-12,36.56
2,2025-05-13,152.83
3,2025-05-14,43.38
4,2025-05-15,37.69
5,2025-05-16,67.06
6,2025-05-17,93.47
7,2025-05-18,125.11
8,2025-05-19,145.97
9,2025-05-20,155.93


In [14]:
# Saving this as a CSV to use on Datawrapper for a histogram chart
by_day.to_csv("../04_tidy_data/chart_01_expenses_by_day.csv", index=False)

#### How much was spent by day and by category?

In [15]:
# Group by date and category and sum the total spent for each category in each of them
by_day_category = df.groupby(["date", "category"])["price_usd_per_capita"].sum().reset_index()

by_day_category

,date,category,price_usd_per_capita
0,2025-05-11,Transportation,1658.25
1,2025-05-12,Food,6.87
2,2025-05-12,Shopping,18.06
3,2025-05-12,Transportation,11.63
4,2025-05-13,Food,13.69
...,...,...,...
68,2025-06-02,Shopping,1.22
69,2025-06-02,Tickets,0.46
70,2025-06-02,Transportation,9.30
71,2025-06-03,Shopping,69.20


We'll pivot this table to get a dataframe to see if we can turn into a **heatmap** or a **streamflow** on our website.

In [82]:
# Pivot the dataframe to a wide format
by_day_category_wide = by_day_category.pivot(index="date", columns="category", values="price_usd_per_capita")

# Flatten the dataframe
by_day_category_wide = pd.DataFrame(by_day_category_wide.to_records())

# Replace all NAs with 0 (in this case it's fine because it means there were 0 expenses in that day for that category)
by_day_category_wide = by_day_category_wide.fillna(0)

# Create a new column for the total expense in each city
by_day_category_wide["total_expenses"] = by_day_category_wide["Food"] + by_day_category_wide["Hotel"] + by_day_category_wide["Shopping"] + by_day_category_wide["Tickets"] + by_day_category_wide["Tour Agency"] + by_day_category_wide["Transportation"]

# Rename the variables so they are following best practices in tidy data:
by_day_category_wide.rename(columns={ "Food":"food",
                                      "Hotel":"hotel",
                                      "Shopping":"shopping",
                                      "Tickets":"tickets",
                                      "Tour Agency":"tour_agency",
                                      "Transportation":"transportation"},
                            inplace=True)

by_day_category_wide

,date,food,hotel,shopping,tickets,tour_agency,transportation,total_expenses
0,2025-05-11,0.00,0.00,0.00,0.00,0.00,1658.25,1658.25
1,2025-05-12,6.87,0.00,18.06,0.00,0.00,11.63,36.56
2,2025-05-13,13.69,20.28,5.42,2.36,75.00,36.08,152.83
3,2025-05-14,17.50,0.00,15.11,7.99,0.00,2.78,43.38
4,2025-05-15,2.08,0.00,6.04,0.00,0.00,29.57,37.69
5,2025-05-16,55.64,0.00,0.00,6.95,0.00,4.47,67.06
6,2025-05-17,29.51,0.00,2.78,0.00,0.00,61.18,93.47
7,2025-05-18,7.81,97.73,0.46,9.26,0.00,9.85,125.11
8,2025-05-19,17.44,0.00,93.24,30.28,0.00,5.01,145.97
9,2025-05-20,12.23,0.00,23.42,4.17,0.00,116.11,155.93


In [83]:
# We'll need to make it long again (but with the zeros for categories with no expenses on a certain day) in order to use it on Flourish
by_day_category_long = by_day_category_wide.drop(columns=["total_expenses"])

# Rename the column names

by_day_category_long.rename(columns={"food":"Food",
                                     "hotel": "Hotel",
                                     "shopping": "Shopping",
                                      "tickets":"Tickets",
                                      "tour_agency":"Tour Agency",
                                      "transportation":"Transportation"}, inplace=True)

by_day_category_long = pd.melt(by_day_category_long, id_vars=["date"], value_vars=["Food",
                                                                                    "Hotel",
                                                                                    "Shopping",
                                                                                    "Tickets",
                                                                                    "Tour Agency",
                                                                                    "Transportation"])



by_day_category_long

,date,variable,value
0,2025-05-11,Food,0.00
1,2025-05-12,Food,6.87
2,2025-05-13,Food,13.69
3,2025-05-14,Food,17.50
4,2025-05-15,Food,2.08
...,...,...,...
139,2025-05-30,Transportation,0.97
140,2025-05-31,Transportation,284.72
141,2025-06-01,Transportation,6.02
142,2025-06-02,Transportation,9.30


In [84]:
# Saving the long format as a CSV to make a Flourish heatmap
by_day_category_long.to_csv("../04_tidy_data/chart_02_expenses_by_day_and_category.csv", index=False)

And now transforming the values into percentages.

In [19]:
# Divide each column by its total
by_day_category_wide["pct_food"] = (by_day_category_wide["food"] / by_day_category_wide["total_expenses"]).round(3)
by_day_category_wide["pct_hotel"] = (by_day_category_wide["hotel"] / by_day_category_wide["total_expenses"]).round(3)
by_day_category_wide["pct_shopping"] = (by_day_category_wide["shopping"] / by_day_category_wide["total_expenses"]).round(3)
by_day_category_wide["pct_tickets"] = (by_day_category_wide["tickets"] / by_day_category_wide["total_expenses"]).round(3)
by_day_category_wide["pct_tour_agency"] = (by_day_category_wide["tour_agency"] / by_day_category_wide["total_expenses"]).round(3)
by_day_category_wide["pct_transportation"] = (by_day_category_wide["transportation"] / by_day_category_wide["total_expenses"]).round(3)	

# Select only the pct columns and the total
by_day_category_wide_pct = by_day_category_wide[["date",
                                                 "pct_food",
                                                 "pct_hotel",
                                                 "pct_shopping",
                                                 "pct_tickets",
                                                 "pct_tour_agency",
                                                 "pct_transportation",
                                                 "total_expenses"]]

by_day_category_wide_pct

,date,pct_food,pct_hotel,pct_shopping,pct_tickets,pct_tour_agency,pct_transportation,total_expenses
0,2025-05-11,0.000,0.000,0.000,0.000,0.000,1.000,1658.25
1,2025-05-12,0.188,0.000,0.494,0.000,0.000,0.318,36.56
2,2025-05-13,0.090,0.133,0.035,0.015,0.491,0.236,152.83
3,2025-05-14,0.403,0.000,0.348,0.184,0.000,0.064,43.38
4,2025-05-15,0.055,0.000,0.160,0.000,0.000,0.785,37.69
5,2025-05-16,0.830,0.000,0.000,0.104,0.000,0.067,67.06
6,2025-05-17,0.316,0.000,0.030,0.000,0.000,0.655,93.47
7,2025-05-18,0.062,0.781,0.004,0.074,0.000,0.079,125.11
8,2025-05-19,0.119,0.000,0.639,0.207,0.000,0.034,145.97
9,2025-05-20,0.078,0.000,0.150,0.027,0.000,0.745,155.93


In [20]:
# Saving this as a CSV to use on Datawrapper for a streamgraph or a heatmap
by_day_category_wide_pct.to_csv("../04_tidy_data/chart_03_expenses_by_day_and_category_pct.csv", index=False)

## 3- Comparing expenses by city

Let's break down how much we spent by day on average in each city:

In [124]:
# Get the total for each city

# Create a dataframe from df
by_city_expenses = df

# Create a new column called "Location" where Lhasa and Shigatse are replaced as "Tibet"
by_city_expenses["location"] = by_city_expenses["city"]
by_city_expenses.loc[by_city_expenses["city"].str.contains("Lhasa", na=False), "location"] = "Tibet"
by_city_expenses.loc[by_city_expenses["city"].str.contains("Shigatse", na=False), "location"] = "Tibet"

# Create a dataframe with the total for each city
by_city_sum = by_city_expenses.groupby("location")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
by_city_sum

,location,price_usd_per_capita
1,China,1652.21
7,Tibet,1279.69
3,Guangzhou,386.95
0,Beijing,371.23
5,Shanghai,365.53
2,Datong,217.33
6,Suzhou,114.41
4,Mutianyu,65.24


In [122]:
# Get the number of days we spent in each city
by_city_days = by_city_expenses.groupby("location")["date"].nunique().reset_index().sort_values(by="date", ascending=False)
by_city_days

,location,date
0,Beijing,13
7,Tibet,6
2,Datong,3
3,Guangzhou,3
5,Shanghai,3
6,Suzhou,2
1,China,1
4,Mutianyu,1


In [126]:
# Merge these two tables and create a new column with the simple average
by_city = by_city_sum.merge(by_city_days, on="location", how='inner')
by_city["daily_average"] = by_city["price_usd_per_capita"]/by_city["date"]
by_city["daily_average"] = by_city["daily_average"].round(0)

# We'll remove the row that says "China" because it only has the airfare from São Paulo to Beijing
by_city = by_city.query("location != 'China'").sort_values(by="daily_average", ascending=False)
by_city

,location,price_usd_per_capita,date,daily_average
1,Tibet,1279.69,6,213.0
2,Guangzhou,386.95,3,129.0
4,Shanghai,365.53,3,122.0
5,Datong,217.33,3,72.0
7,Mutianyu,65.24,1,65.0
6,Suzhou,114.41,2,57.0
3,Beijing,371.23,13,29.0


In [127]:
# CSV for a bar chart
by_city.to_csv("../04_tidy_data/chart_04_avg_spent_by_city.csv", index=False)

This is a good table for a **bar chart** comparing expenses.
<br>
But we can do better. Maybe a **map** locating each city on the Chinese map and comparing the values with spikes.

In [131]:
# Load JSON file (obtained using geojson.io + mapshaper) with the latitude and longitude of each city
cities_geoloc = pd.read_json("../05_dataviz/spatial_data/china_cities/china_cities.json")
cities_geoloc = cities_geoloc.rename(columns={ "city":"location"})
cities_geoloc.loc[cities_geoloc["location"].str.contains("Lhasa", na=False), "location"] = "Tibet"
cities_geoloc

,type,lon,lat,geometry,location
0,Feature,116.387771,39.906573,"{'type': 'Point', 'coordinates': [116.38777063...",Beijing
1,Feature,113.299184,40.090987,"{'type': 'Point', 'coordinates': [113.29918417...",Datong
2,Feature,120.583297,31.299813,"{'type': 'Point', 'coordinates': [120.58329687...",Suzhou
3,Feature,121.469733,31.225088,"{'type': 'Point', 'coordinates': [121.46973268...",Shanghai
4,Feature,113.271242,23.118739,"{'type': 'Point', 'coordinates': [113.27124226...",Guangzhou
5,Feature,91.120724,29.654492,"{'type': 'Point', 'coordinates': [91.120723605...",Tibet
6,Feature,88.886895,29.255383,"{'type': 'Point', 'coordinates': [88.886894998...",Shigatse
7,Feature,116.626866,40.311496,"{'type': 'Point', 'coordinates': [116.62686645...",Mutianyu


In [132]:
# Merge both tables
by_city_geoloc = by_city.merge(cities_geoloc, on="location", how='inner')
by_city_geoloc = by_city_geoloc[["location",
                                 "daily_average",
                                 "lat",
                                 "lon"
                                ]]
by_city_geoloc

,location,daily_average,lat,lon
0,Tibet,213.0,29.654492,91.120724
1,Guangzhou,129.0,23.118739,113.271242
2,Shanghai,122.0,31.225088,121.469733
3,Datong,72.0,40.090987,113.299184
4,Mutianyu,65.0,40.311496,116.626866
5,Suzhou,57.0,31.299813,120.583297
6,Beijing,29.0,39.906573,116.387771


In [133]:
# CSV to make a map
by_city_geoloc.to_csv("../04_tidy_data/chart_04_map_avg_spent_by_city.csv", index=False)

We can also get a **heatmap** or a **streamgraph** if we get expenses by city and category:

In [28]:
# Getting the data in long format
by_city_category = df.groupby(["city", "category"])["price_usd_per_capita"].sum().reset_index()
by_city_category

,city,category,price_usd_per_capita
0,Beijing,Food,98.86
1,Beijing,Shopping,189.99
2,Beijing,Tickets,13.48
3,Beijing,Transportation,68.90
4,China,Transportation,1652.21
5,Datong,Food,32.71
6,Datong,Hotel,20.28
7,Datong,Shopping,21.15
8,Datong,Tickets,7.99
9,Datong,Tour Agency,75.00


In [29]:
# Pivot the dataframe to a wide format
by_city_category_wide = by_city_category.pivot(index="city", columns="category", values="price_usd_per_capita")

# Flatten the dataframe
by_city_category_wide = pd.DataFrame(by_city_category_wide.to_records())

# Replace all NAs with 0 (in this case it's fine because it means there were 0 expenses in that day for that category)
by_city_category_wide = by_city_category_wide.fillna(0)

# Create a new column for the total expense in each city
by_city_category_wide["total_expenses"] = by_city_category_wide["Food"] + by_city_category_wide["Hotel"] + by_city_category_wide["Shopping"] + by_city_category_wide["Tickets"] + by_city_category_wide["Tour Agency"] + by_city_category_wide["Transportation"]

# We'll remove the row that says "China" because it only has the airfare from São Paulo to Beijing
by_city_category_wide = by_city_category_wide.query("city != 'China'").sort_values(by="total_expenses", ascending=False)

# Rename the variables so they are following best practices in tidy data:
by_city_category_wide.rename(columns={ "Food":"food",
                                      "Hotel":"hotel",
                                      "Shopping":"shopping",
                                      "Tickets":"tickets",
                                      "Tour Agency":"tour_agency",
                                      "Transportation":"transportation"},
                            inplace=True)

by_city_category_wide

,city,food,hotel,shopping,tickets,tour_agency,transportation,total_expenses
4,Lhasa,23.20,0.00,38.84,44.45,702.92,430.79,1240.20
3,Guangzhou,11.81,63.98,1.22,17.54,0.00,292.40,386.95
0,Beijing,98.86,0.00,189.99,13.48,0.00,68.90,371.23
6,Shanghai,36.55,67.36,116.66,34.45,0.00,110.51,365.53
2,Datong,32.71,20.28,21.15,7.99,75.00,60.20,217.33
8,Suzhou,2.39,30.37,3.24,9.26,0.00,69.15,114.41
5,Mutianyu,7.50,0.00,2.78,27.78,0.00,27.18,65.24
7,Shigatse,12.42,0.00,27.07,0.00,0.00,0.00,39.49


In [30]:
# CSV for a heatmap or streamgraph
by_city_category_wide.to_csv("../04_tidy_data/chart_05_avg_spent_by_city_and_category.csv", index=False)

Getting the percentages:

In [31]:
# Divide each column by its total
by_city_category_wide["pct_food"] = (by_city_category_wide["food"] / by_city_category_wide["total_expenses"]).round(3)
by_city_category_wide["pct_hotel"] = (by_city_category_wide["hotel"] / by_city_category_wide["total_expenses"]).round(3)
by_city_category_wide["pct_shopping"] = (by_city_category_wide["shopping"] / by_city_category_wide["total_expenses"]).round(3)
by_city_category_wide["pct_tickets"] = (by_city_category_wide["tickets"] / by_city_category_wide["total_expenses"]).round(3)
by_city_category_wide["pct_tour_agency"] = (by_city_category_wide["tour_agency"] / by_city_category_wide["total_expenses"]).round(3)
by_city_category_wide["pct_transportation"] = (by_city_category_wide["transportation"] / by_city_category_wide["total_expenses"]).round(3)	

# Select only the pct columns and the total
by_city_category_wide_pct = by_city_category_wide[["city",
                                                 "pct_food",
                                                 "pct_hotel",
                                                 "pct_shopping",
                                                 "pct_tickets",
                                                 "pct_tour_agency",
                                                 "pct_transportation",
                                                 "total_expenses"]]

by_city_category_wide_pct

,city,pct_food,pct_hotel,pct_shopping,pct_tickets,pct_tour_agency,pct_transportation,total_expenses
4,Lhasa,0.019,0.000,0.031,0.036,0.567,0.347,1240.20
3,Guangzhou,0.031,0.165,0.003,0.045,0.000,0.756,386.95
0,Beijing,0.266,0.000,0.512,0.036,0.000,0.186,371.23
6,Shanghai,0.100,0.184,0.319,0.094,0.000,0.302,365.53
2,Datong,0.151,0.093,0.097,0.037,0.345,0.277,217.33
8,Suzhou,0.021,0.265,0.028,0.081,0.000,0.604,114.41
5,Mutianyu,0.115,0.000,0.043,0.426,0.000,0.417,65.24
7,Shigatse,0.315,0.000,0.685,0.000,0.000,0.000,39.49


In [32]:
# CSV for a heatmap or streamgraph
by_city_category_wide_pct.to_csv("../04_tidy_data/chart_06_avg_spent_by_city_and_category_pct.csv", index=False)

## 4- Breakdown by city

### 4.1- Beijing

In [33]:
# All expenses from Beijing
df_beijing = df[df["city"] == "Beijing"]

In [34]:
# Top expenses from Beijing
top5_beijing = df_beijing.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_beijing

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
147,2025-05-30,Beijing,Silk scarves,Paula,credit card,Shopping,1145.00,159.03,79.51
161,2025-06-03,Beijing,Taobao and Meituan,Renata,apps,Shopping,840.47,116.73,58.37
42,2025-05-16,Beijing,Dinner at Migas (Spanish),Diva,apps,Food,645.00,89.58,44.79
61,2025-05-17,Beijing,Dinner,Tica,apps,Food,303.00,42.08,21.04
2,2025-05-12,Beijing,Necklace and earrings Carol + gift earring fro...,Carol,apps,Shopping,170.00,23.61,11.81


In [35]:
# Expenses in Beijing by category
category_beijing = df_beijing.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_beijing

,category,price_usd_per_capita
1,Shopping,189.99
0,Food,98.86
3,Transportation,68.90
2,Tickets,13.48


In [36]:
# CSV for a donut or bar chart
category_beijing.to_csv("../04_tidy_data/chart_07_category_beijing.csv", index=False)

### 4.2- Datong

In [37]:
# All expenses from Datong
df_datong = df[df["city"] == "Datong"]

In [38]:
# Top expenses from Datong
top5_datong = df_datong.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_datong

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
15,2025-05-13,Datong,Datong tourism package,Paula,credit card,Tour Agency,1080.0,150.00,75.00
12,2025-05-13,Datong,Train from Beijing to Datong,Renata,apps,Transportation,378.0,52.50,26.25
38,2025-05-15,Datong,Train from Datong to Beijing,Renata,apps,Transportation,366.0,50.83,25.42
22,2025-05-13,Datong,Da Tong Weidu International Hotel,Diva,apps,Hotel,292.0,40.56,20.28
33,2025-05-14,Datong,Lunch at Yunjinhui,Diva,apps,Food,232.0,32.22,16.11


In [39]:
# Expenses in Datong by category
category_datong = df_datong.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_datong

,category,price_usd_per_capita
4,Tour Agency,75.00
5,Transportation,60.20
0,Food,32.71
2,Shopping,21.15
1,Hotel,20.28
3,Tickets,7.99


In [40]:
# CSV for a donut or bar chart
category_datong.to_csv("../04_tidy_data/chart_08_category_datong.csv", index=False)

### 4.3- Suzhou

In [41]:
# All expenses from Suzhou
df_suzhou = df[df["city"] == "Suzhou"]

In [42]:
# Top expenses from Suzhou
top5_suzhou = df_suzhou.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_suzhou

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
55,2025-05-17,Suzhou,Train from Beijing to Suzhou,Renata,apps,Transportation,1224.0,170.00,56.67
59,2025-05-18,Suzhou,HanTin Premium Hotel,Renata,apps,Hotel,656.0,91.11,30.37
63,2025-05-18,Suzhou,Entrance Humble Administrator's Garden,Carol,apps,Tickets,200.0,27.78,9.26
62,2025-05-18,Suzhou,Train from Suzhou to Shanghai,Renata,apps,Transportation,168.0,23.33,7.78
54,2025-05-17,Suzhou,Didi to Beijingnan station,Diva,apps,Transportation,47.0,6.53,3.26


In [43]:
# Expenses in Suzhou by category
category_suzhou = df_suzhou.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_suzhou

,category,price_usd_per_capita
4,Transportation,69.15
1,Hotel,30.37
3,Tickets,9.26
2,Shopping,3.24
0,Food,2.39


In [44]:
# CSV for a donut or bar chart
category_suzhou.to_csv("../04_tidy_data/chart_09_category_suzhou.csv", index=False)

### 4.4- Shanghai

In [45]:
# All expenses from Shanghai
df_shanghai = df[df["city"] == "Shanghai"]

In [46]:
# Top expenses from Shanghai
top5_shanghai = df_shanghai.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_shanghai

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
87,2025-05-20,Shanghai,Flight from Shanghai to Beijing,Renata,apps,Transportation,1460.0,202.78,101.39
84,2025-05-19,Shanghai,Uniqlo haul,Paula,credit card,Shopping,1162.0,161.39,80.69
70,2025-05-18,Shanghai,Homeinn Hotel,Renata,apps,Hotel,970.0,134.72,67.36
74,2025-05-19,Shanghai,Oriental Pearl Tower ticket,Diva,apps,Tickets,256.0,35.56,17.78
78,2025-05-19,Shanghai,MAP Museum of Art of Pudong,Diva,apps,Tickets,180.0,25.00,12.50


In [47]:
# Expenses in Shanghai by category
category_shanghai = df_shanghai.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_shanghai

,category,price_usd_per_capita
2,Shopping,116.66
4,Transportation,110.51
1,Hotel,67.36
0,Food,36.55
3,Tickets,34.45


In [48]:
# CSV for a donut or bar chart
category_shanghai.to_csv("../04_tidy_data/chart_10_category_shanghai.csv", index=False)

### 4.5- Mutianyu (Great Wall of China)

In [49]:
# All expenses from Shanghai
df_mutianyu = df[df["city"] == "Mutianyu"]

In [50]:
# Top expenses from Shanghai
top5_mutianyu = df_mutianyu.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_mutianyu

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
103,2025-05-21,Mutianyu,Park tickets + shuttle + cable car,Paula,credit card,Tickets,400.00,55.56,27.78
105,2025-05-21,Mutianyu,Didi return,Diva,apps,Transportation,204.20,28.36,14.18
102,2025-05-21,Mutianyu,Didi going,Diva,apps,Transportation,187.27,26.01,13.00
101,2025-05-21,Mutianyu,Lunch Saigon Baimen,Diva,apps,Food,108.00,15.00,7.50
104,2025-05-21,Mutianyu,Hat,Carol,apps,Shopping,40.00,5.56,2.78


In [51]:
# Expenses in Mutianyu by category
category_mutianyu = df_mutianyu.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_mutianyu

,category,price_usd_per_capita
2,Tickets,27.78
3,Transportation,27.18
0,Food,7.50
1,Shopping,2.78


In [52]:
# CSV for a donut or bar chart
category_mutianyu.to_csv("../04_tidy_data/chart_11_category_mutianyu.csv", index=False)

### 4.6- Lhasa

In [53]:
# All expenses from Lhasa
df_lhasa = df[df["city"] == "Lhasa"]
df_lhasa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 112 to 141
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  17 non-null     object 
 1   city                  17 non-null     object 
 2   expense               17 non-null     object 
 3   payment_source        17 non-null     object 
 4   payment_type          17 non-null     object 
 5   category              17 non-null     object 
 6   price                 17 non-null     float64
 7   price_usd             17 non-null     float64
 8   price_usd_per_capita  17 non-null     float64
dtypes: float64(3), object(6)
memory usage: 1.3+ KB


In [54]:
# Top expenses from Lhasa
top5_lhasa = df_lhasa.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_lhasa

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
114,2025-05-24,Lhasa,Tibet tour package,Paula,credit card,Tour Agency,15183.00,2108.75,702.92
112,2025-05-24,Lhasa,Round-trip flight from Beijing to Lhasa,Paula,credit card,Transportation,9169.11,1273.49,424.50
115,2025-05-24,Lhasa,Princess Wejcheng Show,Tica,apps,Tickets,840.00,116.67,38.89
113,2025-05-24,Lhasa,Surprise hot pot lunch,Tica,apps,Food,438.00,60.83,20.28
140,2025-05-29,Lhasa,Shopping at Norbulingka,Diva,apps,Shopping,308.00,42.78,14.26


In [55]:
# Expenses in Lhasa by category
category_lhasa = df_lhasa.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_lhasa

,category,price_usd_per_capita
3,Tour Agency,702.92
4,Transportation,430.79
2,Tickets,44.45
1,Shopping,38.84
0,Food,23.20


In [56]:
# CSV for a donut or bar chart
category_lhasa.to_csv("../04_tidy_data/chart_12_category_lhasa.csv", index=False)

### 4.7- Shigatse

In [57]:
# All expenses from Shigatse
df_shigatse = df[df["city"] == "Shigatse"]
df_shigatse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 123 to 133
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  11 non-null     object 
 1   city                  11 non-null     object 
 2   expense               11 non-null     object 
 3   payment_source        11 non-null     object 
 4   payment_type          11 non-null     object 
 5   category              11 non-null     object 
 6   price                 11 non-null     float64
 7   price_usd             11 non-null     float64
 8   price_usd_per_capita  11 non-null     float64
dtypes: float64(3), object(6)
memory usage: 880.0+ bytes


In [58]:
# Top expenses from Shigatse
top5_shigatse = df_shigatse.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_shigatse

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
127,2025-05-28,Shigatse,Shopping at the Old Market,Carol,apps,Shopping,125.0,17.36,8.68
123,2025-05-27,Shigatse,Tip for Samsara and Kunga,Carol,apps,Shopping,100.0,13.89,6.94
131,2025-05-28,Shigatse,Dinner at Pizza Hut,Diva,apps,Food,57.0,7.92,3.96
124,2025-05-27,Shigatse,Dinner,Carol,apps,Food,52.0,7.22,3.61
126,2025-05-28,Shigatse,Cups,Carol,apps,Shopping,50.0,6.94,3.47


In [59]:
# Expenses in Shigatse by category
category_shigatse = df_shigatse.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_shigatse

,category,price_usd_per_capita
1,Shopping,27.07
0,Food,12.42


In [60]:
# CSV for a donut or bar chart
category_shigatse.to_csv("../04_tidy_data/chart_13_category_shigatse.csv", index=False)

### 4.8- Guangzhou

In [61]:
# All expenses from Guangzhou
df_guangzhou = df[df["city"] == "Guangzhou"]
df_guangzhou.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 148 to 160
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  12 non-null     object 
 1   city                  12 non-null     object 
 2   expense               12 non-null     object 
 3   payment_source        12 non-null     object 
 4   payment_type          12 non-null     object 
 5   category              12 non-null     object 
 6   price                 12 non-null     float64
 7   price_usd             12 non-null     float64
 8   price_usd_per_capita  12 non-null     float64
dtypes: float64(3), object(6)
memory usage: 960.0+ bytes


In [62]:
# Top expenses from Guangzhou
top5_guangzhou = df_guangzhou.sort_values(by="price_usd_per_capita", ascending=False).nlargest(5, "price_usd_per_capita")
top5_guangzhou

,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
149,2025-05-31,Guangzhou,Round-trip flight from Beijing to Guangzhou,Renata,apps,Transportation,6150.0,854.17,284.72
148,2025-05-31,Guangzhou,SunYat Sen University Kaifeng Hotel,Renata,apps,Hotel,1382.0,191.94,63.98
150,2025-06-01,Guangzhou,Pearl River Cruise,Renata,apps,Tickets,369.0,51.25,17.08
151,2025-06-01,Guangzhou,Car from airport to hotel,Renata,apps,Transportation,130.0,18.06,6.02
153,2025-06-01,Guangzhou,Dinner,Diva,apps,Food,120.0,16.67,5.56


In [63]:
# Expenses in Guangzhou by category
category_guangzhou = df_guangzhou.groupby("category")["price_usd_per_capita"].sum().reset_index().sort_values(by="price_usd_per_capita", ascending=False)
category_guangzhou

,category,price_usd_per_capita
4,Transportation,292.40
1,Hotel,63.98
3,Tickets,17.54
0,Food,11.81
2,Shopping,1.22


In [64]:
# CSV for a donut or bar chart
category_guangzhou.to_csv("../04_tidy_data/chart_13_category_guangzhou.csv", index=False)

## 5- Comparing expenses by type of payment

#### How much did we spend using each payment type?

In [65]:
total_by_type = df.groupby("payment_type")["price"].sum().reset_index()
total_by_type = total_by_type.rename(columns={ "price_usd_per_capita" : "total_spent"} )
total_by_type

,payment_type,price
0,apps,24443.66
1,credit card,52149.01


#### How many payments did we make with each type?

In [66]:
count_payment_type = df.groupby("payment_type")["payment_type"].value_counts().reset_index()
count_payment_type = count_payment_type.rename(columns={ "price_usd_per_capita" : "payments_amount"} )
count_payment_type

,payment_type,count
0,apps,154
1,credit card,10


#### What was the average price per capita of expenses from each type of payment?

In [67]:
avg_payment_type = df.groupby("payment_type")["price_usd_per_capita"].mean().reset_index()
avg_payment_type = avg_payment_type.rename(columns={ "price_usd_per_capita" : "avg_spent"} )
avg_payment_type

,payment_type,avg_spent
0,apps,9.057468
1,credit card,305.774000


In [68]:
# Joining all of these calculations (in three steps)
# Step 1: create a list with all the dataframes I want to merge
dfs_to_merge = [total_by_type, count_payment_type, avg_payment_type]

In [69]:
import functools as ft
payments = ft.reduce(lambda left, right: pd.merge(left, right, on="payment_type"), dfs_to_merge)
payments

,payment_type,price,count,avg_spent
0,apps,24443.66,154,9.057468
1,credit card,52149.01,10,305.774000


In [70]:
# Step 2: transpose dataframe

# 2.1: transpose
payments_transposed = payments.T.reset_index()
payments_transposed

,index,0,1
0,payment_type,apps,credit card
1,price,24443.66,52149.01
2,count,154,10
3,avg_spent,9.057468,305.774


In [71]:
# Step 3: make the rows in "payment_type" the new column names

# 3.1: extract the first row into a new object
new_header = payments_transposed.iloc[0]

# 3.2: remove the first row from the transposed dataframe
payments_final = payments_transposed[1:]

# 3.3: apply the values from the new_header object as column names
payments_final.columns = new_header

payments_final

,payment_type,apps,credit card
1,price,24443.66,52149.01
2,count,154,10
3,avg_spent,9.057468,305.774


In [72]:
# Now we just need to finish cleaning and get some insights

# Rename the columns "payment_type" and "credit card" (tidy data best practices)
payments_final = payments_final.rename(columns={ "payment_type" : "calculation",
                                                 "credit card" : "credit_card"
                                               } )

# Transform "apps" and "credit_card" into floats
payments_final["apps"] = payments_final["apps"].astype(float).round(2)
payments_final["credit_card"] = payments_final["credit_card"].astype(float).round(2)

# Now we can get the total and proportion for each payment type
payments_final["total"] = payments_final["apps"] + payments_final["credit_card"]
payments_final["pct_apps"] = (payments_final["apps"] / payments_final["total"]).round(3)
payments_final

,calculation,apps,credit_card,total,pct_apps
1,price,24443.66,52149.01,76592.67,0.319
2,count,154.00,10.00,164.00,0.939
3,avg_spent,9.06,305.77,314.83,0.029


So although 93.9% of our purchases were made using the apps, they only represented 31.9% of the total we spent.

In [73]:
# CSV for a donut or bar chart
payments_final.to_csv("../04_tidy_data/chart_14_expenses_by_payment_type.csv", index=False)

## 5- Curious insights

#### What was our travel calendar?

In [137]:
calendar = df[["date","city"]].drop_duplicates()
calendar

,date,city
0,2025-05-11,Beijing
1,2025-05-12,Beijing
12,2025-05-13,Datong
14,2025-05-13,Beijing
27,2025-05-14,Datong
35,2025-05-15,Beijing
36,2025-05-15,Datong
41,2025-05-16,Beijing
50,2025-05-17,Suzhou
51,2025-05-17,Beijing


In [138]:
# CSV to make a calendar on Flourish
calendar.to_csv("../04_tidy_data/chart_15_calendar.csv", index=False)

#### What did we buy the most?
Making a **word cloud** with this data.

In [74]:
words = pd.Series(df["expense"])
words
word_count = words.str.lower().str.findall(r'\b\w+\b').explode().value_counts().reset_index()
word_count.head(20)

,expense,count
0,to,39
1,didi,26
2,the,23
3,at,18
4,from,14
5,hotel,13
6,lunch,12
7,subway,12
8,shopping,12
9,and,10


In [85]:
# CSV to create a word cloud
word_count.to_csv("../04_tidy_data/chart_15_word_cloud.csv", index=False)

In [ ]:
#### What is the most expensive means of transportation?

In [104]:
transportation = df[df["category"].isin(["Transportation"])].reset_index().sort_values(by="price_usd_per_capita", ascending=False)
transportation = transportation[transportation["city"] != "China"]
transportation

,index,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
43,112,2025-05-24,Lhasa,Round-trip flight from Beijing to Lhasa,Paula,credit card,Transportation,9169.11,1273.49,424.50
50,149,2025-05-31,Guangzhou,Round-trip flight from Beijing to Guangzhou,Renata,apps,Transportation,6150.00,854.17,284.72
33,87,2025-05-20,Shanghai,Flight from Shanghai to Beijing,Renata,apps,Transportation,1460.00,202.78,101.39
22,55,2025-05-17,Suzhou,Train from Beijing to Suzhou,Renata,apps,Transportation,1224.00,170.00,56.67
7,12,2025-05-13,Datong,Train from Beijing to Datong,Renata,apps,Transportation,378.00,52.50,26.25
18,38,2025-05-15,Datong,Train from Datong to Beijing,Renata,apps,Transportation,366.00,50.83,25.42
39,105,2025-05-21,Mutianyu,Didi return,Diva,apps,Transportation,204.20,28.36,14.18
38,102,2025-05-21,Mutianyu,Didi going,Diva,apps,Transportation,187.27,26.01,13.00
25,62,2025-05-18,Suzhou,Train from Suzhou to Shanghai,Renata,apps,Transportation,168.00,23.33,7.78
52,154,2025-06-02,Beijing,Didi from the airport,Renata,apps,Transportation,110.00,15.28,7.64


In [105]:
transportation["transportation_type"] = translated["expense"].str.replace("Transport", "Transportation")
translated["category"] = translated["category"].str.replace("Shopping/Gifts", "Shopping")
translated["category"] = translated["category"].str.replace("Agency", "Tour Agency")

NameError: name 'translated' is not defined

In [106]:
# Identify rows containing 'apple' and add 'Fruit' to 'New_Column'
transportation.loc[transportation["expense"].str.contains("Didi", na=False), "transportation_type"] = "Taxi"
transportation.loc[transportation["expense"].str.contains("Taxi", na=False), "transportation_type"] = "Taxi"
transportation.loc[transportation["expense"].str.contains("taxi", na=False), "transportation_type"] = "Taxi"
transportation.loc[transportation["expense"].str.contains("Car", na=False), "transportation_type"] = "Taxi"
transportation.loc[transportation["expense"].str.contains("Subway", na=False), "transportation_type"] = "Subway"
transportation.loc[transportation["expense"].str.contains("subway", na=False), "transportation_type"] = "Subway"
transportation.loc[transportation["expense"].str.contains("Flight", na=False), "transportation_type"] = "Flight"
transportation.loc[transportation["expense"].str.contains("flight", na=False), "transportation_type"] = "Flight"
transportation.loc[transportation["expense"].str.contains("Train", na=False), "transportation_type"] = "Train"
transportation.loc[transportation["expense"].str.contains("train", na=False), "transportation_type"] = "Train"
transportation.loc[transportation["expense"].str.contains("Boat", na=False), "transportation_type"] = "Boat"
transportation.loc[transportation["expense"].str.contains("boat", na=False), "transportation_type"] = "Boat"
transportation.loc[transportation["expense"].str.contains("Bus", na=False), "transportation_type"] = "Bus"
transportation.head(5)

,index,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita,transportation_type
43,112,2025-05-24,Lhasa,Round-trip flight from Beijing to Lhasa,Paula,credit card,Transportation,9169.11,1273.49,424.50,Flight
50,149,2025-05-31,Guangzhou,Round-trip flight from Beijing to Guangzhou,Renata,apps,Transportation,6150.00,854.17,284.72,Flight
33,87,2025-05-20,Shanghai,Flight from Shanghai to Beijing,Renata,apps,Transportation,1460.00,202.78,101.39,Flight
22,55,2025-05-17,Suzhou,Train from Beijing to Suzhou,Renata,apps,Transportation,1224.00,170.00,56.67,Train
7,12,2025-05-13,Datong,Train from Beijing to Datong,Renata,apps,Transportation,378.00,52.50,26.25,Train


In [110]:
transportation_type = transportation.groupby("transportation_type")["price_usd_per_capita"].mean().round(2).reset_index()
transportation_type = transportation_type.sort_values(by="price_usd_per_capita", ascending=False)
transportation_type

,transportation_type,price_usd_per_capita
2,Flight,270.20
5,Train,29.03
4,Taxi,3.72
1,Bus,2.43
0,Boat,0.64
3,Subway,0.59


In [112]:
# CSV to create a chart
transportation_type.to_csv("../04_tidy_data/chart_16_transportation.csv", index=False)

#### Who was the biggest shopper?
Two interesting purchases stand out.

In [76]:
shopping = df[df["category"].isin(["Shopping"])].reset_index().sort_values(by="price_usd_per_capita", ascending=False)
shopping

,index,date,city,expense,payment_source,payment_type,category,price,price_usd,price_usd_per_capita
16,84,2025-05-19,Shanghai,Uniqlo haul,Paula,credit card,Shopping,1162.00,161.39,80.69
38,147,2025-05-30,Beijing,Silk scarves,Paula,credit card,Shopping,1145.00,159.03,79.51
41,161,2025-06-03,Beijing,Taobao and Meituan,Renata,apps,Shopping,840.47,116.73,58.37
35,140,2025-05-29,Lhasa,Shopping at Norbulingka,Diva,apps,Shopping,308.00,42.78,14.26
0,2,2025-05-12,Beijing,Necklace and earrings Carol + gift earring fro...,Carol,apps,Shopping,170.00,23.61,11.81
22,108,2025-05-22,Beijing,Little shopping at museum,Diva,apps,Shopping,165.00,22.92,11.46
42,162,2025-06-03,Beijing,Foot massage,Carol,apps,Shopping,156.00,21.67,10.83
20,99,2025-05-20,Shanghai,Chocolates and Inácio’s toy car,Diva,apps,Shopping,128.00,17.78,8.89
30,127,2025-05-28,Shigatse,Shopping at the Old Market,Carol,apps,Shopping,125.00,17.36,8.68
19,97,2025-05-20,Shanghai,Little shopping,Diva,apps,Shopping,120.00,16.67,8.33


In [77]:
# CSV to create a word cloud
shopping.to_csv("../04_tidy_data/chart_16_shopping.csv", index=False)